In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
urlist=['https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/page-blocks-1-3_vs_4.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/ecoli4.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/poker-8_vs_6.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/winequality-red-8_vs_6.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/yeast-2_vs_4.dat']
option=int(input("Enter 0 for page-blocks\nEnter 1 for ecoli\nEnter 2 for poker\nEnter 3 for winequality\nEnter 4 for yeast\n"))


Enter 0 for page-blocks
Enter 1 for ecoli
Enter 2 for poker
Enter 3 for winequality
Enter 4 for yeast
 1


In [3]:
import pandas as pd
url=urlist[option]
data = pd.read_csv(url, sep=",", header='infer' )
data

,0.49,0.29,0.48,0.50,0.56,0.24,0.35,negative
0,0.07,0.40,0.48,0.5,0.54,0.35,0.44,negative
1,0.56,0.40,0.48,0.5,0.49,0.37,0.46,negative
2,0.59,0.49,0.48,0.5,0.52,0.45,0.36,negative
3,0.23,0.32,0.48,0.5,0.55,0.25,0.35,negative
4,0.67,0.39,0.48,0.5,0.36,0.38,0.46,negative
...,...,...,...,...,...,...,...,...
330,0.74,0.56,0.48,0.5,0.47,0.68,0.30,negative
331,0.71,0.57,0.48,0.5,0.48,0.35,0.32,negative
332,0.61,0.60,0.48,0.5,0.44,0.39,0.38,negative
333,0.59,0.61,0.48,0.5,0.42,0.42,0.37,negative


In [4]:
t=()
t=data.shape
X = data.values[:,0:(t[1]-1)].astype(float)
Y = data.values[:,(t[1]-1)]
print(X)

[[0.07 0.4  0.48 ... 0.54 0.35 0.44]
 [0.56 0.4  0.48 ... 0.49 0.37 0.46]
 [0.59 0.49 0.48 ... 0.52 0.45 0.36]
 ...
 [0.61 0.6  0.48 ... 0.44 0.39 0.38]
 [0.59 0.61 0.48 ... 0.42 0.42 0.37]
 [0.74 0.74 0.48 ... 0.31 0.53 0.52]]


In [5]:
if option==0 or option==3 or option==2 :
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    print(X)

In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)
ysi=pd.Series(encoded_Y) 
ysi.value_counts()

0    313
2     20
1      2
dtype: int64

In [7]:
if option==1:
    yk=[]
    for i in encoded_Y:
        if i==2:
            yk.append(1)
        else:
            yk.append(0)
    encoded_Y = np.asarray(yk, dtype=np.float32)
    encoded_Y.shape

In [8]:
if option==0:
    rs=1
    ep=10
    ne=2000
if option==1:
    rs=0
    ep=10
    ne=700
if option==2:
    rs=1
    ep=30
    ne=1500
if option==3:
    rs=4
    ep=30
    ne=1500    
if option==4:
    rs=1
    ep=10
    ne=1000    
    
    

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,encoded_Y, test_size=0.2, random_state=rs)

In [10]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))
from imblearn.over_sampling import SMOTE
X_train_res,y_train_res = SMOTE().fit_resample(X_train,y_train)

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 18
Before OverSampling, counts of label '0': 250 

After OverSampling, the shape of train_X: (500, 7)
After OverSampling, the shape of train_y: (500,) 

After OverSampling, counts of label '1': 250
After OverSampling, counts of label '0': 250


In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
from sklearn.metrics import f1_score
from statistics import stdev

2.4.1


In [12]:
def callf1(xx,yy,xt,yt):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                  loss='mean_absolute_error',
                  metrics=['accuracy'])
    model.fit(xx, yy , epochs=ep)
    ls=[]
    test_loss, test_acc = model.evaluate(xt,  yt, verbose=2)
    print('\nTest accuracy:', test_acc)
    tr_loss, tr_acc = model.evaluate(xx,  yy, verbose=2)
    ls.append(test_acc)
    ls.append(tr_acc)
    ypr=model.predict(xt)
    ypr=(ypr>0.5)*1
    ypre= np.ravel(ypr)
    ls.append(f1_score(yt, ypre))
    return ls

In [13]:
sta=[]
ste=[]
sf =[]
for i in range(30):
    r=callf1(X_train, y_train.ravel(),X_test,y_test.ravel())
    ste.append(r[0])
    sta.append(r[1])
    sf.append(r[2])

Epoch 1/10
9/9 [==============================] - 1s 4ms/step - loss: 0.1109 - accuracy: 0.9308
Epoch 2/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0761 - accuracy: 0.9327
Epoch 3/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0648 - accuracy: 0.9448
Epoch 4/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0674 - accuracy: 0.9359
Epoch 5/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0782 - accuracy: 0.9242
Epoch 6/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0626 - accuracy: 0.9408
Epoch 7/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0765 - accuracy: 0.9236
Epoch 8/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0661 - accuracy: 0.9361
Epoch 9/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0727 - accuracy: 0.9281
Epoch 10/10
9/9 [==============================] - 0s 3ms/step - loss: 0.0602 - accuracy: 0.9428
3/3 - 0s - loss: 0.0375 - accuracy: 0.9

In [14]:
sum(sf)/30

0.0

In [15]:
sum(sta)/30

0.9328358173370361

In [16]:
sum(ste)/30

0.9701492786407471

In [17]:
max(sf)

0.0

In [18]:
max(sta)

0.9328358173370361

In [19]:
max(ste)

0.9701492786407471

In [20]:
stdev(sf)

0.0

In [21]:
stdev(sta)

0.0

In [22]:
stdev(ste)

0.0

In [23]:
sta2=[]
ste2=[]
sf2 =[]
for i in range(30):
    r=callf1(X_train_res,y_train_res.ravel(),X_test,y_test.ravel())
    ste2.append(r[0])
    sta2.append(r[1])
    sf2.append(r[2])

Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4173 - accuracy: 0.5597
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 0.2194 - accuracy: 0.9061
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1695 - accuracy: 0.9718
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1453 - accuracy: 0.9778
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1467 - accuracy: 0.9820
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1597 - accuracy: 0.9769
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1525 - accuracy: 0.9868
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1354 - accuracy: 0.9879
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1303 - accuracy: 0.9930
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1350 - accuracy: 0.9929
3/3 - 0s - loss: 0.

In [24]:
sum(sf2)/30

0.5398412698412698

In [25]:
sum(sta2)/30

0.990666663646698

In [26]:
sum(ste2)/30

0.9552238643169403

In [27]:
max(sf2)

0.6666666666666666

In [28]:
max(sta2)

0.9959999918937683

In [29]:
max(ste2)

0.9701492786407471

In [30]:
stdev(sf2)

0.08969486621576842

In [31]:
stdev(sta2)

0.003536342509014137

In [32]:
stdev(ste2)

0.006788945594420794

In [75]:
t2=X_train.shape

In [76]:
X_oversampled=X_train_res[(t2[0]):]

In [77]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [78]:
torch.cuda.empty_cache()

In [79]:
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )
class Generator(nn.Module):

    def __init__(self, z_dim=t2[1], im_dim=t2[1], hidden_dim=128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):

        return self.gen
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)        
    )
class Discriminator(nn.Module):
    def __init__(self, im_dim=t2[1], hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):

        return self.disc(image)
    
    def get_disc(self):

        return self.dis
X_oversampled = torch.from_numpy(X_oversampled)
criterion = nn.BCEWithLogitsLoss()
n_epochs = ne
z_dim = t2[1]
batch_size = 128
lr = 0.00001
display_step = 1
device = 'cuda'
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
def get_disc_loss(gen, disc, criterion, real, device):

    fake = gen(X_oversampled.float().to(device))
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    fake_images = gen(X_oversampled.float().to(device))
    
    disc_fake_pred = disc(fake_images)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss
y_tr=y_train.ravel()
li=[]
for i in range(len(y_tr)):
    if int(y_tr[i])==1:
        li.append(X_train[i])
X_real=np.array(li)
t3=X_real.shape
li2=[1]*(t3[0])
y_real=np.array(li2)
y_real.shape
from torch.utils.data import TensorDataset, DataLoader
tensor_x = torch.Tensor(X_real) 
tensor_y = torch.Tensor(y_real)
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
dataloader = DataLoader(
    my_dataset,
    batch_size=batch_size,
    shuffle=True)
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True 
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, device)

        # Update gradients#
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step()        
        #### END CODE HERE ####

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1, step 1: Generator loss: 1.3941781520843506, discriminator loss: 1.3859213590621948


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2, step 2: Generator loss: 0.696718156337738, discriminator loss: 0.6927323341369629


...

Epoch 697, step 697: Generator loss: 0.6936326026916504, discriminator loss: 0.6942541599273682


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 698, step 698: Generator loss: 0.6935052275657654, discriminator loss: 0.6942701935768127


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 699, step 699: Generator loss: 0.6933797597885132, discriminator loss: 0.694280207157135


In [80]:
res=gen(X_oversampled.float().to(device))

In [81]:
res

tensor([[0.7653, 0.7514, 0.4293,  ..., 0.8395, 0.5913, 0.1878],
        [0.7614, 0.7198, 0.4217,  ..., 0.7606, 0.5863, 0.1968],
        [0.7854, 0.7900, 0.4345,  ..., 0.6746, 0.5517, 0.1653],
        ...,
        [0.7632, 0.5724, 0.4999,  ..., 0.6952, 0.5786, 0.1854],
        [0.7889, 0.6171, 0.4627,  ..., 0.7024, 0.6256, 0.1580],
        [0.7346, 0.6524, 0.4347,  ..., 0.6828, 0.5282, 0.2040]],
       device='cuda:0', grad_fn=<SigmoidBackward>)

In [82]:
X_oversampled

tensor([[0.6374, 0.6812, 0.4800,  ..., 0.6618, 0.4085, 0.2829],
        [0.6145, 0.6878, 0.4800,  ..., 0.6765, 0.3989, 0.3176],
        [0.6815, 0.8285, 0.4800,  ..., 0.7530, 0.5443, 0.3599],
        ...,
        [0.6300, 0.5310, 0.4800,  ..., 0.7613, 0.4418, 0.2800],
        [0.6561, 0.5765, 0.4800,  ..., 0.7226, 0.3494, 0.2439],
        [0.7055, 0.7600, 0.4800,  ..., 0.8364, 0.4936, 0.2809]],
       dtype=torch.float64)

In [83]:
fres=res.cpu().detach().numpy()
fres.shape

(232, 7)

In [84]:
tu=X_train.shape

In [85]:
fin=np.concatenate((X_train_res[:(tu[0])], fres), axis=0)

In [86]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [87]:
y_train_res.shape

(500,)

In [88]:
Xn,yn=shuffle_in_unison(fin, y_train_res)

In [89]:
ste3=[]
sta3=[]
sf3=[]
for i in range(30):
    r=callf1(Xn, yn.ravel(),X_test,  y_test.ravel())
    ste3.append(r[0])
    sta3.append(r[1])
    sf3.append(r[2])

Epoch 1/10
16/16 [==============================] - 0s 2ms/step - loss: 0.4109 - accuracy: 0.5669
Epoch 2/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1762 - accuracy: 0.9390
Epoch 3/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1378 - accuracy: 0.9729
Epoch 4/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1319 - accuracy: 0.9624
Epoch 5/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1147 - accuracy: 0.9779
Epoch 6/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1023 - accuracy: 0.9737
Epoch 7/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1061 - accuracy: 0.9610
Epoch 8/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1029 - accuracy: 0.9745
Epoch 9/10
16/16 [==============================] - 0s 2ms/step - loss: 0.0980 - accuracy: 0.9762
Epoch 10/10
16/16 [==============================] - 0s 2ms/step - loss: 0.1017 - accuracy: 0.9808
3/3 - 0s - loss: 0.

In [90]:
sum(sf3)/30

0.722698412698413

In [91]:
sum(sta3)/30

0.9808666706085205

In [92]:
sum(ste3)/30

0.9766169289747874

In [93]:
max(sf3)

0.8

In [94]:
max(sta3)

0.9879999756813049

In [95]:
max(ste3)

0.9850746393203735

In [96]:
stdev(sf3)

0.091960713064945

In [97]:
stdev(sta3)

0.0046292412733403

In [98]:
stdev(ste3)

0.010132997904346259

In [99]:
save=1